<a href="https://colab.research.google.com/github/christoforou/CUS1166_Project_Template/blob/master/Computer_vision/Learning_activity_code/04_TransferLearning_example_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook is part the of Dr. Christoforos Christoforou's course materials. You may not, nor may you knowingly allow others to reproduce or distribute lecture notes, course materials or any of their derivatives without the instructor's express written consent.

## Tranfer Learing and Practical Consideration Traning CNNs - Sample code
This code showcase the use of **Transfer Learning**  to use an existing pre-trained model and use it to classify images at a different domain. The code is explained as part of the corresponding course lecture, refer to the lecture video for details explanation of the various sections on the example. In brief, the example implements the following:
- Downloads a sample dataset from kaggle (you need a API key to download the data). 
- Defines a generator to load batches for a training set. 
- Creates network that utilizes transfer learning for object recognition. 
- Train the network.
- Using the network to perform object recognition in the new image domain. 

In [1]:
import pandas as pd
import numpy as np
import os
import tensorflow.keras
import matplotlib.pyplot as plt

from tensorflow.keras.layers import Dense,GlobalAveragePooling2D
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet import preprocess_input
from tensorflow.keras.applications.mobilenet import decode_predictions
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

from skimage import io 
import matplotlib.pyplot as plt
from skimage import io
from skimage.transform import resize

## Download the dataset

In [ ]:
# Upload the kaggle API key of your account to download the data.  
!pip install -q kaggle
from google.colab import files
files.upload() 
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d pranavraikokte/covid19-image-dataset
!mkdir sample_data/covid
!unzip covid19-image-dataset.zip -d sample_data/covid

In [ ]:
img = io.imread('/content/sample_data/covid/Covid19-dataset/train/Covid/01.jpeg')
plt.imshow(img)

## Creating the model using transfer learning.

In [ ]:
# Impoort the mobilenet model but discard the last layer
base_model = MobileNet(weights='imagenet', include_top=False,input_shape=(224,224,3))

augmented_model = Sequential()

augmented_model.add(base_model)  # Add the base model.
augmented_model.add(GlobalAveragePooling2D())
augmented_model.add(Dense(units=1024, activation='relu'))
augmented_model.add(Dense(units=512,activation='relu'))
augmented_model.add(Dense(units=3, activation='softmax'))

In [ ]:
# Specify Trainable parameters 
for i, layer in enumerate(augmented_model.layers):
   if (i==0):
     layer.trainable = False 
     print(f"Layer {i} (Non-trainable): {layer}")
   else:
      layer.trainable = True 
      print(f"Layer {i} (Trainable): {layer}")


## Define data generator 


In [ ]:
data_path = '/content/sample_data/covid/Covid19-dataset/train'
data_path_valid = '/content/sample_data/covid/Covid19-dataset/test'

In [ ]:
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    preprocessing_function=preprocess_input) 

In [ ]:
train_generator = train_datagen.flow_from_directory(data_path,
                                                    target_size=(224,224),
                                                    batch_size=32,
                                                    class_mode='categorical')

validation_generator = train_datagen.flow_from_directory(data_path_valid,
                                                    target_size=(224,224),
                                                    batch_size=32,
                                                    class_mode='categorical')

## Compile and Train the model 

In [ ]:
# Now we can prepaire our model for training

augmented_model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

step_size_train = train_generator.n / train_generator.batch_size
step_size_valid = validation_generator.n / validation_generator.batch_size

hist = augmented_model.fit(train_generator, steps_per_epoch=step_size_train, epochs=30,validation_data=validation_generator, validation_steps=step_size_valid)

## Apply the model and visualize results 

In [ ]:
data_path_predict = '/content/sample_data/covid/Covid19-dataset/test/'
image_files = [
    data_path_predict + "Covid/0100.jpeg",
    data_path_predict + "Covid/0102.jpeg",
    data_path_predict + "Covid/0105.png",
    data_path_predict + "Covid/0119.jpeg",
    data_path_predict + "Covid/COVID-00003b.jpg",
    data_path_predict + "Viral Pneumonia/0101.jpeg",
    data_path_predict + "Viral Pneumonia/0102.jpeg",
    data_path_predict + "Viral Pneumonia/0105.jpeg",
    data_path_predict + "Viral Pneumonia/0109.jpeg",
    data_path_predict + "Viral Pneumonia/0112.jpeg",
    data_path_predict + "Normal/0103.jpeg",
    data_path_predict + "Normal/0106.jpeg",
    data_path_predict + "Normal/0107.jpeg",
    data_path_predict + "Normal/0109.jpeg",
    data_path_predict + "Normal/0110.jpeg",               
]

In [ ]:
IMAGE_WIDTH = 224
IMAGE_HEIGHT = 224
IMAGE_CHANNELS = 3

In [ ]:
mg_index = 4
pred_all = []
for img_index in range(len(image_files)):
  print(image_files[img_index])
  image = io.imread(image_files[img_index]) 
  image = resize(image,output_shape=(IMAGE_WIDTH,IMAGE_HEIGHT,IMAGE_CHANNELS), preserve_range=True)

  x = np.copy(image)               # create a copy of the image 
  x = np.expand_dims(x, axis=0)    # Add an observation dimension  
  x = preprocess_input(x)          # pre-process image, based on mobileNet pre-processing. 
  y_pred = augmented_model.predict(x)        # Apply Mobile Net to make predictions.
  pred_all.append(y_pred)
  
  print(f"Image index {img_index} predicted class : {np.argmax(y_pred)}")

In [ ]:
class_labels = ["Covid","Normal", "Pneumonia"]
true_target = [0,0,0,0,0,2,2,2,2,2,1,1,1,1,1]

fig = plt.figure(figsize=(10,10))
for img_index in range(len(image_files)):
  #print(image_files[img_index])
  image = io.imread(image_files[img_index]) 
  image = resize(image,output_shape=(IMAGE_WIDTH,IMAGE_HEIGHT,IMAGE_CHANNELS), preserve_range=False)
  y_pred = np.argmax(pred_all[img_index])
  y_true = true_target[img_index] 

  plt.subplot(5,4,img_index+1)
  plt.imshow(image)
  plt.title(f"T: {class_labels[y_true]} | P: {class_labels[y_pred]}")

fig.tight_layout()


Copyright Statement: Copyright © 2020 Christoforou. The materials provided by the instructor of this course, including this notebook, are for the use of the students enrolled in the course. Materials are presented in an educational context for personal use and study and should not be shared, distributed, disseminated or sold in print — or digitally — outside the course without permission. You may not, nor may you knowingly allow others to reproduce or distribute lecture notes, course materials as well as any of their derivatives without the instructor's express written consent.